<h2>Initialise files</h2>

In [ ]:
import pandas as pd
import numpy as np
import math
import os
from functools import reduce
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns

Declare variable constants

In [ ]:
FOLER_PATH = 'UOA_raw'
OUTPUT_FILE_PATH = 'uoa_gen'

Parse, combine and output data in all UOA files

In [ ]:
def parse_data_df_to_excel(df): 
    df['Exp Date'] = pd.to_datetime(df['Exp Date']).dt.date
    df['Trade date'] = pd.to_datetime(df['Trade date']).dt.date
    data_df = df.loc[:].dropna()
    data_df = data_df.astype({'DTE': int})
    return data_df.values.tolist()

In [ ]:
def parse_data_headers(df): 
    df.rename(columns={'Time': 'Trade date'}, inplace=True)
    return df.columns.values.tolist()

In [ ]:
import csv

columns_headers = [] 
data = []

for entry in os.listdir(FOLER_PATH):
    file_path = os.path.join(FOLER_PATH, entry)
    print(file_path)
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path)
        column_headers = parse_data_headers(df)
        data.extend(parse_data_df_to_excel(df))

# flatten as single level array 
all_data = []
all_data.extend([column_headers])
all_data.extend(data)

print(all_data[0])
print(all_data[1])

output_file = open(f'{OUTPUT_FILE_PATH}.csv', 'w',  newline='')
writer =  csv.writer(output_file)
writer.writerows(all_data)
output_file.close()

<h2>ETL data</h2>

Display some data for visual check 

In [ ]:
df = pd.read_csv(f'{OUTPUT_FILE_PATH}.csv')
display(df.head())
display(df.info())

<h3>Sort and add useful columns</h3>

"""
add vol * mid 
arrange according to ticker / DTE / type
calculate total call + put 
"""

In [ ]:
df = df.sort_values(['Symbol', 'DTE', 'Type'], ascending=True)
df['Amount'] = df['Volume'] * df['Midpoint']
display(df)

Replace file

In [ ]:
df.to_csv(f'{OUTPUT_FILE_PATH}.csv')

<h3>Daily trade put-call charts</h3>

In [ ]:
daily_group = df.groupby(["Trade date", "Type"]).agg({'Amount': 'sum'})
daily_group = daily_group.reset_index()
daily_group

In [ ]:
def line_plot(data): 
    fig = plt.figure(figsize=(20, 6))
    sns.lineplot(data=data, x="Trade date", y="Amount", hue="Type")
    plt.show()
    
line_plot(daily_group)